Решение задачи РайфайзенБанка по предсказанию цен на недвижимость. Сентябрь 2021 года, команда DataSlon

In [25]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor  #lib for classification
from sklearn.model_selection import train_test_split
import pickle  #for saving model
from catboost import CatBoostRegressor
from catboost import Pool

In [2]:
# подключаем гугл диск на котором данные
from google.colab import drive
drive.mount ('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [4]:
!ls /content/gdrive/'My Drive'/raifhack/data

test.csv  test_submission.csv  train.csv


In [5]:
PATH_TO_DATA = "../content/gdrive/My Drive/raifhack/data"

In [6]:
train_df = pd.read_csv(PATH_TO_DATA+'/train.csv')
train_df.tail(2)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,per_square_meter_price,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
279790,Кемерово,1.0,COL_280529,55.360680,86.081460,5,57,100,134,0,0,0,0,1,16,32,41,0.844577,Кемерово,558662.0,0.341931,0,10,12,20,1,3,5,7,1,4,6,7,1,3,3,1,7,9,0,0,1,1,4,7,0,2,9,12,1,23,32,48,200.109079,2.073608,0,0,0,0.385545,3,14,18,38755.038322,94,24,1649.0,429.0,4.691489,4.125000,1957.425532,1954.625000,Кемеровская область,89.201305,S17667,2020-08-23,110,1
279791,Киров,1.0,COL_280637,58.592915,49.677439,1,23,64,113,0,0,0,0,0,1,11,25,1.317512,Киров,496986.0,0.126115,0,6,18,48,0,0,0,1,0,6,9,12,3,4,5,0,1,2,1,1,3,1,1,4,0,0,4,7,1,10,37,61,306.460323,1.998461,0,0,0,0.148057,4,12,19,41335.014762,249,79,2735.0,766.0,5.069124,4.776119,1972.599078,1971.794118,Кировская область,172.855871,S5611,2020-08-23,110,1


In [7]:
#смотрим размерность данный, 279 ТЫС. СТРОК, 77 СТОЛБЦОВ
train_df.shape

(279792, 77)

In [9]:
#смотрим названия колонок
train_df.columns

Index(['city', 'floor', 'id', 'lat', 'lng', 'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
       'osm_city_nearest_name', 'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075',
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in

In [10]:
#let s count NaNs
train_df.isna().sum().sum()

312480

In [29]:
X_features = ['city','lat', 'lng',
       'osm_amenity_points_in_0.01', 
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist','osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.01',
       'osm_culture_points_in_0.01',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.01',
       'osm_historic_points_in_0.005', 'osm_hotels_points_in_0.01',
       'osm_offices_points_in_0.01','osm_shops_points_in_0.01', 'total_square', 'realty_type',
       'price_type']

In [30]:
# Разделим годы на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(train_df[X_features], 
                                                    train_df['per_square_meter_price'], 
                                                    test_size=0.3,  
                                                    #stratify=y_length, 
                                                    random_state=42)

In [24]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 28 kB/s 


In [31]:
target_col = 'per_square_meter_price'
text_cols = []
categorical_cols = ['city']

train_pool = Pool(
        X_train, 
        y_train, 
        cat_features=categorical_cols,
        text_features=text_cols,
        feature_names=X_features,
)
valid_pool = Pool(
        X_test, 
        y_test, 
        cat_features=categorical_cols,
        text_features=text_cols,
        feature_names=X_features,
)

In [32]:
regr_CB =CatBoostRegressor(iterations= 800,
    learning_rate= 0.015,
    depth=16,
    task_type= 'CPU',   #'GPU',
    early_stopping_rounds= 90,
    #class_weights=[0.95,0.05],
    use_best_model= True,
    random_seed=42,
    verbose= 20
)

In [33]:
regr_CB.fit(train_pool, eval_set=valid_pool,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 169757.3633900	test: 172621.8027528	best: 172621.8027528 (0)	total: 4.36s	remaining: 58m 7s
20:	learn: 145296.7345699	test: 148429.0959744	best: 148429.0959744 (20)	total: 1m 8s	remaining: 42m 14s
40:	learn: 127806.4403586	test: 131305.1767682	best: 131305.1767682 (40)	total: 2m 12s	remaining: 40m 49s
60:	learn: 115419.4829485	test: 119321.1595812	best: 119321.1595812 (60)	total: 3m 16s	remaining: 39m 38s
80:	learn: 106749.4115327	test: 110916.4203188	best: 110916.4203188 (80)	total: 4m 17s	remaining: 38m 3s
100:	learn: 100344.0946353	test: 104911.9142924	best: 104911.9142924 (100)	total: 5m 21s	remaining: 37m 3s
120:	learn: 95693.9364959	test: 100624.3982565	best: 100624.3982565 (120)	total: 6m 24s	remaining: 35m 59s
140:	learn: 92205.9627962	test: 97563.2112398	best: 97563.2112398 (140)	total: 7m 28s	remaining: 34m 56s
160:	learn: 89279.8225770	test: 95075.1681564	best: 95075.1681564 (160)	total: 8m 31s	remaining: 33m 51s
180:	learn: 86788.8805396	test: 92992.3867132	best: 

In [35]:
features_list = list(regr_CB.feature_importances_)
for i in range(len(X_features)):
  print(X_features[i],features_list[i])

city 2.3511719600826146
lat 6.380927770649267
lng 8.853421300353574
osm_amenity_points_in_0.01 1.3187134999683798
osm_building_points_in_0.01 4.748888924473569
osm_catering_points_in_0.001 8.784356141242341
osm_catering_points_in_0.01 1.7442856887662412
osm_city_closest_dist 4.67294513114419
osm_city_nearest_population 9.005780748053093
osm_crossing_closest_dist 5.807423437471979
osm_crossing_points_in_0.001 4.5416249034988585
osm_crossing_points_in_0.01 2.478393375286349
osm_culture_points_in_0.01 2.835541809117371
osm_finance_points_in_0.01 3.897292715414583
osm_healthcare_points_in_0.01 1.7185341206770504
osm_historic_points_in_0.005 3.8658428131410236
osm_hotels_points_in_0.01 1.36587500121734
osm_offices_points_in_0.01 1.276181294480991
osm_shops_points_in_0.01 1.7397468824250721
total_square 13.945981021799543
realty_type 8.621903365840108
price_type 0.04516809489646769


In [36]:
regr_CB.score(X_train,y_train)

0.8442028998486368

In [37]:
regr_CB.score(X_test,y_test)

0.7852241495369973

In [39]:
test_df = pd.read_csv(PATH_TO_DATA+'/test.csv')  #merge 
test_df.tail(2)

,city,floor,id,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,lat,lng,total_square,street,date,realty_type,price_type
2972,Нижний Новгород,NaN,COL_455381,13,70,114,158,0,0,0,0,8,34,45,59,1.030306,Нижний Новгород,1252236.0,0.050913,4,12,22,38,0,5,9,12,0,0,1,1,1,1,2,4,6,11,4,6,7,10,15,20,1,5,7,8,1,14,33,47,1.657414,2.588330,0,0,0,0.032220,7,10,19,199,38,1343.0,370.0,3.829268,4.529412,1938.331361,1946.529412,Нижегородская область,56.328236,43.990039,208.625735,S21753,2020-12-27,10,1
2973,Нижний Новгород,NaN,COL_455397,1,52,111,146,0,0,0,0,1,7,20,28,8.620167,Нижний Новгород,1252236.0,0.054001,1,19,27,39,0,0,4,4,0,3,7,14,1,2,2,2,6,8,0,0,2,2,4,7,0,2,4,8,0,36,69,81,2.114953,1.017884,0,0,0,0.419797,1,18,19,134,50,2932.0,943.0,6.961240,5.440000,1972.892308,1963.940000,Нижегородская область,56.343952,43.869360,232.109058,S28997,2020-12-27,100,1


In [40]:
pred_test = regr_CB.predict(test_df[X_features])
print(pred_test[:10])

[ 30715.55166814  54448.76148842  56130.01095925  90555.95062571
  52057.05344162  59092.40381448  59225.45278455  51901.42039861
 114867.79435717  51077.65963136]


In [41]:
test_df['per_square_meter_price'] = pred_test
test_df.tail(2)

,city,floor,id,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,lat,lng,total_square,street,date,realty_type,price_type,per_square_meter_price
2972,Нижний Новгород,NaN,COL_455381,13,70,114,158,0,0,0,0,8,34,45,59,1.030306,Нижний Новгород,1252236.0,0.050913,4,12,22,38,0,5,9,12,0,0,1,1,1,1,2,4,6,11,4,6,7,10,15,20,1,5,7,8,1,14,33,47,1.657414,2.588330,0,0,0,0.032220,7,10,19,199,38,1343.0,370.0,3.829268,4.529412,1938.331361,1946.529412,Нижегородская область,56.328236,43.990039,208.625735,S21753,2020-12-27,10,1,64648.348422
2973,Нижний Новгород,NaN,COL_455397,1,52,111,146,0,0,0,0,1,7,20,28,8.620167,Нижний Новгород,1252236.0,0.054001,1,19,27,39,0,0,4,4,0,3,7,14,1,2,2,2,6,8,0,0,2,2,4,7,0,2,4,8,0,36,69,81,2.114953,1.017884,0,0,0,0.419797,1,18,19,134,50,2932.0,943.0,6.961240,5.440000,1972.892308,1963.940000,Нижегородская область,56.343952,43.869360,232.109058,S28997,2020-12-27,100,1,54650.691321


In [42]:
#let s save our prediction

test_df[['id','per_square_meter_price']].to_csv('sub01.csv',header=True,index=False)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
!ls

catboost_info  gdrive  sample_data  sub01.csv


In [44]:
from google.colab import files

files.download('sub01.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>